In [ ]:
!git clone https://github.com/neccam/slt.git

Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.


In [ ]:
%cd slt

/content/slt


In [ ]:
!chmod +x data/download.sh
!data/download.sh

--2020-12-26 19:30:29--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  20.3MB/s    in 92s     

2020-12-26 19:32:02 (20.0 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2020-12-26 19:32:02--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130831227 (125M)
Saving to: ‘phoenix14t.pami0.dev’

phoenix14t.pami0.de 100%[===================>] 124.77M  17.9MB/s    in 7.5s    

2020-12-26 19:32:10 (16.7 MB/s) - ‘phoenix14t.pami0.dev’ saved [1308312

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

In [ ]:
!pip -q install -r requirements.txt

     |████████████████████████████████| 112kB 13.8MB/s 
     |████████████████████████████████| 112kB 11.6MB/s 
     |████████████████████████████████| 163kB 18.4MB/s 
     |████████████████████████████████| 399kB 28.1MB/s 
     |████████████████████████████████| 2.3MB 31.7MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 61kB 10.5MB/s 
     |████████████████████████████████| 2.4MB 65.9MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
     |████████████████████████████████| 296kB 74.8MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 92kB 12.7MB/s 
     |████████████████████████████████| 92kB 13.7MB/s 
     |████████████████████████████████| 13.1MB 72.4MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 20.2MB 85.3MB/s 
     

In [ ]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!cp -r ../drive/MyDrive/slt/dev_features.zip data/dev_features.zip
!cp -r ../drive/MyDrive/slt/train_features.zip data/train_features.zip
!cp -r ../drive/MyDrive/slt/test_features.zip data/test_features.zip

!unzip -q data/dev_features.zip -d data
!unzip -q data/train_features.zip -d data
!unzip -q data/test_features.zip -d data

In [ ]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

for s in ['train', 'dev', 'test']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)

  src_features = load_dataset_file(filename)
  for i, src in tqdm(enumerate(src_features)):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)
  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

7096it [01:23, 84.52it/s]
519it [00:06, 82.19it/s]
642it [00:07, 81.67it/s]


In [ ]:
!ls data/PHOENIX2014T

phoenix14t.pami0.dev   phoenix14t.pami0.train  phoenix14t.pami1.test
phoenix14t.pami0.test  phoenix14t.pami1.dev    phoenix14t.pami1.train


In [ ]:
"""
shapes = [new_features.shape[0] for new_features in new_features_list]
new_features_list = np.concatenate(new_features_list, axis=0)

from sklearn.decomposition import PCA

pca = PCA(n_components=256, svd_solver='arpack')
np.random.seed(2020)

random_idx = np.random.choice(np.arange(new_features_list.shape[0]), 20000)
pca.fit(new_features_list[random_idx])
new_features_list = pca.transform(new_features_list)

new_features_list_reshape = []
start = 0
for shape in shapes:
  end=start+shape
  new_features_list_reshape.append(new_features_list[start: end])
  start=end
"""

In [ ]:
"""
for s in ['train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)

  src_features = load_dataset_file(filename)
  new_features_list = []
  for i, src in tqdm(enumerate(src_features)):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    src['sign'] = torch.cat((src['sign'], torch.Tensor(new_features_list_reshape[i])), 1)
with gzip.open(filename, "wb") as f:
  pickle.dump(src_features, f)
  """

7096it [00:02, 2585.45it/s]


In [ ]:
"""
for s in ['dev', 'test']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)

  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    new_features = pca.transform(new_features)
    src['sign'] = torch.cat((src['sign'], torch.Tensor(new_features)), 1)
  with gzip.open(filename, "wb") as f:
    pickle.dump(src_features, f)
  del src_features
"""

In [ ]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(2)]

list_doc['data']['feature_size'] = [1024, 1024]
list_doc['model']['model_dir'] = "hidden_features_fusion"

In [ ]:
list_doc

{'data': {'data_path': './data/',
  'dev': ['PHOENIX2014T/phoenix14t.pami0.dev',
   'PHOENIX2014T/phoenix14t.pami1.dev'],
  'feature_size': [1024, 1024],
  'gls': 'gloss',
  'level': 'word',
  'max_sent_length': 400,
  'random_dev_subset': -1,
  'random_train_subset': -1,
  'sgn': 'sign',
  'test': ['PHOENIX2014T/phoenix14t.pami0.test',
   'PHOENIX2014T/phoenix14t.pami1.test'],
  'train': ['PHOENIX2014T/phoenix14t.pami0.train',
   'PHOENIX2014T/phoenix14t.pami1.train'],
  'txt': 'text',
  'txt_lowercase': True,
  'version': 'phoenix_2014_trans'},
 'model': {'bias_initializer': 'zeros',
  'decoder': {'dropout': 0.1,
   'embeddings': {'activation_type': 'softsign',
    'dropout': 0.1,
    'embedding_dim': 512,
    'norm_type': 'batch',
    'scale': False},
   'ff_size': 2048,
   'hidden_size': 512,
   'num_heads': 8,
   'num_layers': 3,
   'type': 'transformer'},
  'embed_init_gain': 1.0,
  'embed_initializer': 'xavier',
  'encoder': {'dropout': 0.1,
   'embeddings': {'activation_type': 

In [ ]:
with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

In [ ]:
!python -m signjoey train configs/sign.yaml

2020-12-26 19:54:06.356273: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-26 19:54:12.507622: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-26 19:54:12.527919: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-26 19:54:12.580623: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-26 19:54:12.581260: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 836.37GiB/s
2020-12-26 19:54:12.581299: I tensorflow/stream_executor/platform/defau

In [ ]:
!cp -r ./sign_sample_model "../drive/MyDrive/slt/models/hidden_features_fusion" 